In [ ]:
import os
import pandas as pd
from textblob import *
import nltk
import numpy as np
from sklearn import tree
import openpyxl
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import *
'''
    ISOT
'''


read_file = r"../data/Des_fake_news/WELFAKE_HF.csv"
read_sheet = "valid"
write_file = r"../data/Des_fake_news/WELFAKE_PROCESSED.csv"

#data = pd.read_excel(read_file, read_sheet)
data = pd.read_csv(read_file)

data = data.dropna(axis=0)
data

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
...,...,...,...,...
72129,72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0
72130,72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1
72131,72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0
72132,72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,0


In [12]:
#remove nonASCIICharacters

data["title"] = data.apply(lambda x: "".join([i for i in x["title"] if ord(i) <= 127]), axis=1)



data["text"] = data.apply(lambda x: "".join([i for i in x["text"] if ord(i) <= 127]), axis=1)


In [13]:
# SKIP FOR WELFAKE
'''
def to_class(s):
    if s == "real":
        return 1
    return 0
'''

'\ndef to_class(s):\n    if s == "real":\n        return 1\n    return 0\n'

In [14]:
#data["flag"] = data.apply(lambda x: to_class(x["flag"]), axis=1)
#data

In [15]:
data["text_tb_pol"] = data.apply(lambda x: TextBlob(x["text"]).polarity, axis=1)
data["text_tb_sub"] = data.apply(lambda x: TextBlob(x["text"]).subjectivity, axis=1)

data["title_tb_pol"] = data.apply(lambda x: TextBlob(x["title"]).polarity, axis=1)
data["title_tb_sub"] = data.apply(lambda x: TextBlob(x["title"]).subjectivity, axis=1)
data

,Unnamed: 0,title,text,label,text_tb_pol,text_tb_sub,title_tb_pol,title_tb_sub
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,0.033103,0.296038,0.080000,0.320000
2,2,UNBELIEVABLE! OBAMAS ATTORNEY GENERAL SAYS MOS...,"Now, most of the demonstrators gathered last ...",1,0.258929,0.400595,0.121875,0.625000
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,0.113865,0.384704,0.000000,0.500000
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,0.056373,0.391967,-0.287879,0.484848
5,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1,-0.079803,0.427094,-0.400000,0.450000
...,...,...,...,...,...,...,...,...
4025,4025,"Rose Evansky, a Pioneer in Womens Hairstyling,...","Rose Evansky, a British hairdresser who libera...",0,0.116051,0.396710,0.368182,0.702273
4026,4026,"Informed Rant: The Violence of Borders, Racist...","Informed Rant: The Violence of Borders, Racist...",1,-0.013890,0.385033,0.200000,0.300000
4027,4027,Baking Soda & Coconut Oil Can Kill Cancer: Eye...,Humans Are Free Baking Soda & Coconut Oil Can ...,1,0.108825,0.507881,0.000000,0.000000
4028,4028,Telefonica suspends customer service operation...,GUATEMALA CITY (Reuters) - The Guatemalan unit...,0,-0.051389,0.400000,0.000000,0.000000


In [16]:
analyzer = SentimentIntensityAnalyzer()

data["title_vader_scores"] = data.apply(lambda x: analyzer.polarity_scores(x["title"]), axis=1)
data["title_vader_comp"] = data.apply(lambda x: x["title_vader_scores"]["compound"], axis=1)
data["title_vader_neg"] = data.apply(lambda x: x["title_vader_scores"]["neg"], axis=1)
data["title_vader_neu"] = data.apply(lambda x: x["title_vader_scores"]["neu"], axis=1)
data["title_vader_pos"] = data.apply(lambda x: x["title_vader_scores"]["pos"], axis=1)
data = data.drop(["title_vader_scores"], axis=1)

data["text_vader_scores"] = data.apply(lambda x: analyzer.polarity_scores(x["text"]), axis=1)
data["text_vader_comp"] = data.apply(lambda x: x["text_vader_scores"]["compound"], axis=1)
data["text_vader_neg"] = data.apply(lambda x: x["text_vader_scores"]["neg"], axis=1)
data["text_vader_neu"] = data.apply(lambda x: x["text_vader_scores"]["neu"], axis=1)
data["text_vader_pos"] = data.apply(lambda x: x["text_vader_scores"]["pos"], axis=1)
data = data.drop(["text_vader_scores"], axis=1)

In [17]:
def discrete_vader(neg, neu, pos):
    return np.argmax([neg, neu, pos])
    
def discrete_textblob(pol):
    if pol <= -0.33:
        return 0
    elif pol > -0.33 and pol < 0.33:
        return 1
    else:
        return 2

def discrete_textblob_sub(pol):
    if pol < 0.5:
        return 0
    return 1

In [18]:

data["title_vader_class"] = data.apply(lambda x: discrete_vader(x["title_vader_neg"], x["title_vader_neu"], x["title_vader_pos"]), axis=1)
data["text_vader_class"] = data.apply(lambda x: discrete_vader(x["text_vader_neg"], x["text_vader_neu"], x["text_vader_pos"]), axis=1)

data["text_tb_pol_class"] = data.apply(lambda x: discrete_textblob(x["text_tb_pol"]), axis=1)
data["text_tb_sub_class"] = data.apply(lambda x: discrete_textblob_sub(x["text_tb_sub"]), axis=1)

data["title_tb_pol_class"] = data.apply(lambda x: discrete_textblob(x["title_tb_pol"]), axis=1)
data["title_tb_sub_class"] = data.apply(lambda x: discrete_textblob_sub(x["title_tb_sub"]), axis=1)
data


,Unnamed: 0,title,text,label,text_tb_pol,text_tb_sub,title_tb_pol,title_tb_sub,title_vader_comp,title_vader_neg,...,text_vader_comp,text_vader_neg,text_vader_neu,text_vader_pos,title_vader_class,text_vader_class,text_tb_pol_class,text_tb_sub_class,title_tb_pol_class,title_tb_sub_class
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,0.033103,0.296038,0.080000,0.320000,-0.6081,0.278,...,-0.9926,0.111,0.809,0.080,1,1,1,0,1,0
2,2,UNBELIEVABLE! OBAMAS ATTORNEY GENERAL SAYS MOS...,"Now, most of the demonstrators gathered last ...",1,0.258929,0.400595,0.121875,0.625000,0.7755,0.000,...,0.7351,0.050,0.746,0.204,1,1,1,0,1,1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,0.113865,0.384704,0.000000,0.500000,0.4767,0.000,...,0.9993,0.029,0.839,0.132,1,1,1,0,1,1
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,0.056373,0.391967,-0.287879,0.484848,-0.5719,0.209,...,-0.9081,0.079,0.872,0.049,1,1,1,0,1,0
5,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1,-0.079803,0.427094,-0.400000,0.450000,-0.5093,0.239,...,-0.9255,0.155,0.732,0.113,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4025,4025,"Rose Evansky, a Pioneer in Womens Hairstyling,...","Rose Evansky, a British hairdresser who libera...",0,0.116051,0.396710,0.368182,0.702273,0.0000,0.000,...,0.9166,0.063,0.861,0.076,1,1,1,0,2,1
4026,4026,"Informed Rant: The Violence of Borders, Racist...","Informed Rant: The Violence of Borders, Racist...",1,-0.013890,0.385033,0.200000,0.300000,-0.8885,0.538,...,-0.9810,0.089,0.835,0.076,0,1,1,0,1,0
4027,4027,Baking Soda & Coconut Oil Can Kill Cancer: Eye...,Humans Are Free Baking Soda & Coconut Oil Can ...,1,0.108825,0.507881,0.000000,0.000000,-0.8779,0.532,...,-0.9959,0.145,0.766,0.089,0,1,1,1,1,0
4028,4028,Telefonica suspends customer service operation...,GUATEMALA CITY (Reuters) - The Guatemalan unit...,0,-0.051389,0.400000,0.000000,0.000000,-0.6705,0.360,...,-0.9666,0.142,0.819,0.039,1,1,1,0,1,0


In [40]:
#data.to_csv(r"../data/Des_fake_news/ISOT_PROCESSED.csv", index=False)

In [19]:
'''
    Begin Neural Net for news text
'''

#data = pd.read_csv(r"../data/Des_fake_news/ISOT_PROCESSED.csv")
'''text only'''
features = data[["text_tb_pol", "text_vader_pos", "text_vader_neg", "text_vader_neu"]].to_numpy()


import tensorflow as tf
from sklearn.model_selection import train_test_split

imdb_model = tf.keras.models.load_model(r"SA_imdb/IMDB_NN.keras")
imdb_sentiment = np.argmax(imdb_model.predict(features), axis=-1)

tweets_model = tf.keras.models.load_model(r"SA_tweets/TWEETS_NN.keras")
tweets_sentiment = np.argmax(tweets_model.predict(features), axis=-1)

data["text_NN_imdb"] = imdb_sentiment
data["text_NN_tweets"] = tweets_sentiment

#data.to_csv(r"../data/Des_fake_news/ISOT_PROCESSED.csv", index=False)

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [20]:
'''
    Begin Neural Net for news title
'''


'''text only'''
features = data[["title_tb_pol", "title_vader_pos", "title_vader_neg", "title_vader_neu"]].to_numpy()


import tensorflow as tf
from sklearn.model_selection import train_test_split

imdb_model_two = tf.keras.models.load_model(r"SA_imdb/IMDB_NN.keras")
imdb_sentiment_two = np.argmax(imdb_model_two.predict(features), axis=-1)

tweets_model_two = tf.keras.models.load_model(r"SA_tweets/TWEETS_NN.keras")
tweets_sentiment_two = np.argmax(tweets_model_two.predict(features), axis=-1)

data["title_NN_imdb"] = imdb_sentiment_two
data["title_NN_tweets"] = tweets_sentiment_two

#data.to_csv(r"../data/Des_fake_news/ISOT_PROCESSED.csv", index=False)

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [21]:
'''
    Begin log regression for text
'''
from sklearn.linear_model import LogisticRegression
import pickle

texts = data[["text_tb_pol", "text_vader_pos", "text_vader_neg", "text_vader_neu"]].to_numpy()

tweet_text_log:  LogisticRegression = None
with open(r"SA_tweets/TWEETS_LOG.pkl", "rb") as file:
    tweet_text_log = pickle.load(file)

imdb_text_log:  LogisticRegression = None
with open(r"SA_imdb/IMDB_LOG.pkl", "rb") as file:
    imdb_text_log = pickle.load(file)

data["text_log_imdb"] = imdb_text_log.predict(texts)
data["text_log_tweets"] = imdb_text_log.predict(texts)





In [22]:
'''
    Begin log regression for title
'''
from sklearn.linear_model import LogisticRegression
import pickle

titles = data[["title_tb_pol", "title_vader_pos", "title_vader_neg", "title_vader_neu"]].to_numpy()


tweet_text_log:  LogisticRegression = None
with open(r"SA_tweets/TWEETS_LOG.pkl", "rb") as file:
    tweet_text_log = pickle.load(file)

imdb_text_log:  LogisticRegression = None
with open(r"SA_imdb/IMDB_LOG.pkl", "rb") as file:
    imdb_text_log = pickle.load(file)

data["title_log_imdb"] = imdb_text_log.predict(texts)
data["title_log_tweets"] = imdb_text_log.predict(texts)


data.to_csv(write_file, index=False)

In [23]:
data.to_csv(write_file)